In [18]:
import pandas as pd
import numpy as np
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM



df1=pd.read_csv("Indian-Male-Names.csv")
df2=pd.read_csv("Indian-Female-Names.csv")

In [19]:
df=pd.concat([df1['name'],df2['name']],ignore_index=True,axis=0,names=['Name'])

In [20]:
df=df.str.encode('utf-8').str.decode('ascii', 'ignore')

In [21]:
df.replace(regex=True, inplace=True, to_replace=r'[^abcdefghijklmnopqrstuvwxyz ]', value=r' ')

In [22]:
df=df.dropna()

In [23]:
df.isnull().sum()

0

In [170]:
nameList=df.sample(20000).values
nameString='\n'.join(nameList)

In [68]:
chars = list(set(nameString))

In [69]:
len(chars)

27

In [74]:
characters = sorted(list(set(nameString)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [75]:
n_to_char

{0: ' ',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

In [131]:
X = []
Y = []
length = len(nameString)
seq_length = 50
for i in range(0, length-seq_length, 1):
    sequence = nameString[i:i + seq_length]
    label =nameString[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [132]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [138]:
model = Sequential()
model.add(LSTM(30, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(30))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [139]:
model.fit(X_modified, Y_modified, epochs=2, batch_size=500)

Epoch 1/2
51007/51007 [==============================] - 21s 405us/step - loss: 2.9457
Epoch 2/2
51007/51007 [==============================] - 19s 378us/step - loss: 2.8296


In [140]:
string_mapped = X[99]
# generating characters
for i in range(seq_length):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))
    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]


ValueError: Error when checking input: expected lstm_25_input to have shape (50, 1) but got array with shape (51, 1)

In [ ]:
full_string = [n_to_char[value] for value in string_mapped]
#combining text#combinin 
txt=""
for char in full_string:
    txt = txt+char
txt

In [ ]:
full_string

In [171]:
text=nameString

maxlen=30
step=3
sentences=[]
next_chars=[]
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('Number of sequences : ',len(sentences))

chars=sorted(list(set(text)))
print('Unique characters: ',len(chars))

char_indices=dict((char,chars.index(char)) for char in chars)

print("Vectorization")
x=np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)

for i,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1
    






Number of sequences :  67867
Unique characters:  28
Vectorization


In [172]:
import keras
from keras import layers


model=keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))
optimizer=keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)          

In [173]:
def sample(preds,temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)


In [174]:
import random
import sys

for item in range(1,5):
    print()
    print('Try ',item)
    model.fit(x,y,batch_size=128,epochs=1)
    start_index=random.randint(0,len(text)-maxlen-1)
    generated_text=text[start_index:start_index+maxlen]
    print('---Generating with seed : "'+ generated_text +' "')    
    for temperature in [0.2,0.5,1.0,1.2]:
        print('---randomness :',temperature)
        sys.stdout.write(generated_text)    
        for i in range(400):
            sampled=np.zeros((1,maxlen,len(chars)))
            for t,char in enumerate(generated_text):
                sampled[0,t,char_indices[char]]=1

            preds=model.predict(sampled,verbose=0)[0]
            next_index=sample(preds,temperature)
            next_char=chars[next_index]

            generated_text+=next_char
            generated_text=generated_text[1:]
            sys.stdout.write(next_char)


Try  1
Epoch 1/1
67867/67867 [==============================] - 42s 620us/step - loss: 2.0853
---Generating with seed : "ushpa
saraswati thapa
mohmmd s "
---temperature : 0.2
ushpa
saraswati thapa
mohmmd singh
anita
amar
anita
anita
ramita
anita
anita
anita
anita
raman
kamari
ramu
anita
anita
anita
anita
raju
sanita
amita
anita
anita
anita
ramita
anita
sanita
raman
anita
anita
anita
amari
anita
raju
sanita
amari
anita
anita
raja
sanita
anita
anita
anita
anita
anita
anita
amita
ramita
anita
anita
sanita
raman
rajeen
raju
anita
anita
anita
sanita
amar kumar
arani
raju
parana
sanita
anita
amari
raju
a---temperature : 0.5
rana
sanita
anita
amari
raju
amari
madha
rameen
kamal
parani
kamari
aman kumar
ashira
arun
sairan
anita
deepa
arul
kaman
aashi
anashi
reshi
param
parita
parana
deepak
ramul
priyaka
amari prayag
sadeep
pasha
rekha
deepak rama
rajaya
uman
sanita
aarak kumar
sameet
kamal
raju
parti
asha
bala
sakhi devi
arita
priyanka
panu
amari
anuri
sameet
anita
preeti
aram
kamal
kamari de